In [1]:
import sys
import numpy as np
from ctypes import *
import ctypes
import torch
import math
import random

if "/home/yskim/projects/sparse-bfp" not in sys.path:
    sys.path.append("/home/yskim/projects/sparse-bfp")
from util.reprod_util import set_reproducibility

from util.bfp.bfp_config import BfpConfig
from util.bfp.cuda_bfp_wrapper import CudaBfpWrapper

c_lib = ctypes.CDLL("/home/yskim/projects/sparse-bfp/util/sample/lib_sample.so")

In [5]:
def load_inout(path_root, phase, layer_index, n_index, k_cnt, m_index):
    path = f"{path_root}/{phase}-{layer_index}-fp"

    lhs = np.load(f"{path}-lhs.npy", allow_pickle=False)
    rhs = np.load(f"{path}-rhs.npy", allow_pickle=False)
    output = np.matmul(lhs, rhs)
    # print(f"lhs: {lhs.shape} X rhs: {rhs.shape} = output: {output.shape}")
    
    # lhs = lhs[n_index,0:k_cnt]
    # rhs = rhs[0:k_cnt,m_index]
    # output = np.dot(lhs, rhs)
    # print(f"lhs: {lhs.shape} X rhs: {rhs.shape} = output: {output.shape}")

    return lhs, rhs, output

log_path = "/home/yskim/projects/sparse-bfp/resnet50-inout-log/epoch0-lr0.001"

lhs, rhs, output = load_inout(log_path, "bwd-a", 30, 0, 2, 0)

lhs = torch.tensor(lhs).to("cuda")
rhs = torch.tensor(rhs).to("cuda")

print(f"lhs: {lhs.shape}, rhs: {rhs.shape}")

BfpConfig.bfp_M_Bit = 8

lhs_wrapper = CudaBfpWrapper(fp_tensor_shape=lhs.shape, use_flex_bfp=False)
lhs_wrapper.run_convert_fp_to_bfp(src_tensor=lhs, is_stochastic_rounding=True)

rhs_wrapper = CudaBfpWrapper(fp_tensor_shape=rhs.shape, use_flex_bfp=False)
rhs_wrapper.run_convert_fp_to_bfp(src_tensor=rhs, is_stochastic_rounding=True)



# output_torch = torch.dot(lhs, rhs)
# print(lhs)
# print(rhs)
# print(output)
# print(output_torch.item())

# fwd_lhs = np.load(f"{log_path}/fwd-1-fp-lhs.npy", allow_pickle=False)
# fwd_rhs = np.load(f"{log_path}/fwd-1-fp-rhs.npy", allow_pickle=False)

# print(fwd_lhs[0][0:16])
# print(fwd_rhs[0][0:16])

# np.load(f"{log_path}/bwd-w-0-fp-lhs.npy", allow_pickle=False)
# np.load(f"{log_path}/bwd-w-0-fp-rhs.npy", allow_pickle=False)

# print(np.load(f"{log_path}/bwd-a-1-fp-lhs.npy", allow_pickle=False).shape)
# print(np.load(f"{log_path}/bwd-a-1-fp-rhs.npy", allow_pickle=False).shape)



lhs: torch.Size([512, 1024]), rhs: torch.Size([1024, 256])


In [6]:
# for i in range(10):
#     print(lhs_wrapper.bfp_E[i,:])

print("lhs")
mean_E = torch.mean(lhs_wrapper.bfp_E.cpu().type(torch.float), dim=1)
print(mean_E.shape)
print(torch.min(mean_E))
print(torch.max(mean_E))

print("rhs")
mean_E = torch.mean(rhs_wrapper.bfp_E.cpu().type(torch.float), dim=1)
print(mean_E.shape)
print(torch.min(mean_E))
print(torch.max(mean_E))
# lhs_wrapper.bfp_E.shape

lhs
torch.Size([512])
tensor(-17.0156)
tensor(-12.5312)
rhs
torch.Size([1024])
tensor(-4.2500)
tensor(-3.6875)


In [ ]:
lhs_E = lhs_wrapper.bfp_E
rhs_E = rhs_wrapper.bfp_E
M = lhs_E.shape[0]
K = lhs_E.shape[1]
N = rhs_E.shape[1]
E_sum = torch.zeros(size=(M, N, K), dtype=torch.int32)

for m in range(M):
    for n in range(N):
        E_sum[m,n,:] = lhs_E[m,:] + rhs[:,n]

In [8]:
for m in range(M):
    for n in range(N):
        # print(torch.mean(E_sum[m,n].type(torch.float)), end=", ")
        # print(torch.min(E_sum[m,n].type(torch.float)), end=", ")
        # print(torch.max(E_sum[m,n].type(torch.float)))